#Fine-tune AraBERT with Fast-BERT 


##Get Requirements

In [1]:
#!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 -f https://download.pytorch.org/whl/torch_stable.html


In [2]:
!git clone https://github.com/aub-mind/arabert
!pip install PyArabic farasapy fast-bert

Cloning into 'arabert'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 595 (delta 37), reused 43 (delta 29), pack-reused 535
Receiving objects: 100% (595/595), 9.14 MiB | 31.40 MiB/s, done.
Resolving deltas: 100% (338/338), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 7.9 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 4.9 MB 66.4 MB/s 
     |████████████████████████████████| 13.1 MB 59.8 MB/s 
     |████████████████████████████████| 3.0 MB 51.7 MB/s 
     |████████████████████████████████| 212 kB 81.8 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 125 kB 72.1 MB/s 
     |████████████████████████████████| 4.0 MB 59.3 MB/s 
     |████████████████████████████████| 880 kB 

##Prepare the data

In [3]:
import pandas as pd
#from py4j.java_gateway import JavaGateway
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess import ArabertPreprocessor
from sklearn.model_selection import train_test_split

# !pkill "java"
# gateway = JavaGateway.launch_gateway(classpath='./FarasaSegmenterJar.jar')
# farasa_segmenter = gateway.jvm.com.qcri.farasa.segmenter.Farasa()

farasa_segmenter = FarasaSegmenter(interactive=True)

df_A = pd.read_csv('restaurant1.csv',header=0)
df_A

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:25<00:00, 9.41MiB/s]


[2022-08-16 15:56:21,945 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


,sentence,polarity,category,aspect
0,نموت علي اكل تونسي محرحر,Positive,FOOD#QUALITY,اكل
1,شعب هريسه اثبت وجود,Positive,GENERAL,NaN
2,انا فلسطي بفتخر انو تونس عرب مثلي تحيه لبلد را...,Positive,GENERAL,NaN
3,والله اطيب واذكي شعب ومضياف وشعب راقي تونس بوا...,Positive,GENERAL,NaN
4,والله هذا شاب تونسي رجال وكريم وطيب و ابن ناس ...,Positive,GENERAL,NaN
...,...,...,...,...
4212,مضر,Negative,RESTAURANT#cleanliness,نظافة
4213,ياسر مسخ,Negative,RESTAURANT#cleanliness,نظافة
4214,ملا تبلبيز,Negative,RESTAURANT#cleanliness,نظافة
4215,بنينه برشا و نظيفه انظف من محل عصريه نظيفه من ...,Positive,FOOD#QUALITY,اكل


In [4]:
import numpy as np
#df_A['sentiment']= df_A['polarity']

df_A['polarity'] = np.array(list(map(lambda x: 1 if x=="Positive" else 0, df_A['polarity'])))
df_A

,sentence,polarity,category,aspect
0,نموت علي اكل تونسي محرحر,1,FOOD#QUALITY,اكل
1,شعب هريسه اثبت وجود,1,GENERAL,NaN
2,انا فلسطي بفتخر انو تونس عرب مثلي تحيه لبلد را...,1,GENERAL,NaN
3,والله اطيب واذكي شعب ومضياف وشعب راقي تونس بوا...,1,GENERAL,NaN
4,والله هذا شاب تونسي رجال وكريم وطيب و ابن ناس ...,1,GENERAL,NaN
...,...,...,...,...
4212,مضر,0,RESTAURANT#cleanliness,نظافة
4213,ياسر مسخ,0,RESTAURANT#cleanliness,نظافة
4214,ملا تبلبيز,0,RESTAURANT#cleanliness,نظافة
4215,بنينه برشا و نظيفه انظف من محل عصريه نظيفه من ...,1,FOOD#QUALITY,اكل


In [5]:
df_AJGT = df_A[["sentence", "polarity"]]
df_AJGT

,sentence,polarity
0,نموت علي اكل تونسي محرحر,1
1,شعب هريسه اثبت وجود,1
2,انا فلسطي بفتخر انو تونس عرب مثلي تحيه لبلد را...,1
3,والله اطيب واذكي شعب ومضياف وشعب راقي تونس بوا...,1
4,والله هذا شاب تونسي رجال وكريم وطيب و ابن ناس ...,1
...,...,...
4212,مضر,0
4213,ياسر مسخ,0
4214,ملا تبلبيز,0
4215,بنينه برشا و نظيفه انظف من محل عصريه نظيفه من ...,1


In [6]:
df_AJGT['sentence'] = df_AJGT['sentence'].apply(lambda x: ArabertPreprocessor(x))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
df_AJGT['polarity']=df_AJGT['polarity'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.2,random_state=42)
!mkdir data1

In [9]:
train_AJGT.to_csv("data1/train.csv",index=True,columns=train_AJGT.columns,sep=',',header=True)
test_AJGT.to_csv("data1/dev.csv",index=True,columns=test_AJGT.columns,sep=',',header=True)

In [10]:
with open('data1/labels.csv','w') as f:
    f.write("\n".join(df_AJGT['polarity'].unique()))

##Create a DataBunch Object:
see https://github.com/kaushaltrivedi/fast-bert#text-classification

In [11]:
from fast_bert.data_cls import BertDataBunch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')



Downloading:   0%|          | 0.00/637 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/700k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
databunch = BertDataBunch('./data1/', './data1/',
                          tokenizer=tokenizer,
                          train_file='train.csv',
                          val_file='dev.csv',
                          label_file='labels.csv',
                          text_col='sentence',
                          label_col='polarity',
                          batch_size_per_gpu=16,
                          max_seq_length=256,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert',
                          )

##Create the Learner Object
see https://github.com/kaushaltrivedi/fast-bert#2-create-a-learner-object

In [13]:
import logging
import torch

from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

!mkdir 'output'

In [14]:

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='aubmindlab/bert-base-arabert',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='output',
						finetuned_wgts_path=None,
						warmup_steps=30,
						multi_gpu=False,
						is_fp16=False,
						multi_label=False,
						logging_steps=0)

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

##Start Training and Validating

In [15]:
learner.fit(epochs=5,
			lr=2e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_linear",
			optimizer_type="adamw")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(1055, 0.6285330454320139)

##You can see the output using tensorboard

In [16]:
#%load_ext tensorboard
#%tensorboard --logdir './output/tensorboard'

In [17]:
from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
#from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [18]:
from functools import partial

metrics = []
#metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'F1', 'function': partial(fbeta, beta=1)})
metrics.append({'name': 'accuracy_single', 'function': accuracy_multilabel})


In [20]:
#Predictions
#Call predict_batch method on the learner object that contains the trained model.

texts = [
  "نموت علي اكل تونسي محرحر",
  "ياسر مسخ"
]

predictions = learner.predict_batch(texts)
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor('aubmindlab/bert-base-arabert','./data1/')

# Single prediction
single_prediction = predictor.predict("نموت علي اكل تونسي محرحر")

# Batch predictions
texts = ["نموت علي اكل تونسي محرحر",
  "ياسر مسخ"
]

multiple_predictions = predictor.predict(texts)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [21]:
multiple_predictions

[('0', 0.5093144178390503), ('1', 0.4906855821609497)]

In [23]:
single_prediction = predictor.predict("ياسر مسخ")
single_prediction

[('1', 0.5884835124015808), ('0', 0.4115164577960968)]